# Satellite Monthly: Global timeseries

## Import libraries

Switch warnings off for better readability.

In [ ]:
import warnings

import pandas as pd
from c3s_eqc_automatic_quality_control import diagnostics, download, plot

warnings.filterwarnings("ignore")

## Define request

In [ ]:
collection_id = "satellite-aerosol-properties"

year_start = 2018
year_stop = 2019

request = {
    "format": "zip",
    "time_aggregation": "monthly_average",
    "variable": "aerosol_optical_depth",
    "sensor_on_satellite": "slstr_on_sentinel_3a",
    "algorithm": "ens",
    "year": [str(year) for year in range(year_start, year_stop + 1)],
    "month": [f"{month:02d}" for month in range(1, 12 + 1)],
    "version": "v2.1",
}

## Define transform function:

`source_to_time` is a temporary workaround.
The harmonisation of satellite data has not been implemented in the toolbox yet.
Therefore, the toolbox is not able to concatenate satellite data.

In EQC we concatenate satellite data along the dimension `source`, which corresponds to the name of the source file.
Then, we use the name of the source file to infer the `time` dimension.
Unfortunately, there are several naming conventions and this step cannot be automatised.

In [ ]:
def source_to_time_monthly(ds):
    # Naming convention: YYYYMM-*.nc
    ds["source"] = pd.to_datetime(ds["source"].str.slice(None, 6), format="%Y%m")
    return ds.rename(source="time")


def source_to_time_monthly_and_spatial_weighted_mean(ds):
    return diagnostics.spatial_weighted_mean(source_to_time_monthly(ds))


def source_to_time_monthly_and_spatial_weighted_std(ds):
    return diagnostics.spatial_weighted_std(source_to_time_monthly(ds))

## Compute spatially-weighted global mean timeseries

In [ ]:
ds_mean = download.download_and_transform(
    collection_id,
    request,
    transform_func=source_to_time_monthly_and_spatial_weighted_mean,
    split_all=True,
)

## Compute spatially-weighted global std timeseries

In [ ]:
ds_std = download.download_and_transform(
    collection_id,
    request,
    transform_func=source_to_time_monthly_and_spatial_weighted_std,
    split_all=True,
)

## Plot and save figure

In [ ]:
filename = "_".join(["global-timeseries", collection_id, request["time_aggregation"]])
title = filename.replace("-", " ").replace("_", " ").title()
fig = plot.shaded_std("AOD550", ds_mean, ds_std, title=title)
fig.show()
fig.write_image(filename + ".png")